# proofread_nuclei
Here, we first mainly proofread the `'nuc_xyz'` column using Neuroglancer.

In [ ]:
import numpy as np
import pyperclip
import pandas as pd
import pyperclip
from cloudvolume import CloudVolume, view, Bbox
from nglui import statebuilder,annotation,easyviewer,parser
from nglui.statebuilder import *
from nglui.nglite import *
import json
import sys
import os
from datetime import datetime
from caveclient import CAVEclient
from ..lib import get_cv_path
from fanc import rootID_lookup as IDlook

In [ ]:
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)

In [ ]:
df = pd.read_csv('../Output/soma_info_Aug2021ver5.csv', header=0)

## 1. Create Neuroglancer links
You can create various types of annotation in Neuroglancer. Here, we used a line type and connected `'nuc_xyz'` and `'soma_xyz'` (i.e., 17,076 lines in total), and check each of them. See [this example](https://github.com/seung-lab/NeuroglancerAnnotationUI/blob/master/examples/statebuilder_examples.ipynb) as well.

In [ ]:
# we only need xyz for nuclei/somas..., and make them into lines
xyz_df = df.reindex(columns=['nuc_xyz', 'soma_xyz'])
nuc_xyz_df = df['nuc_xyz'].str.strip('()').str.split(',',expand=True)
soma_xyz_df = df['soma_xyz'].str.strip('()').str.split(',',expand=True)
xyz_df['nuc_xyz'] = nuc_xyz_df.astype(int).values.tolist()
xyz_df['soma_xyz'] = soma_xyz_df.astype(int).values.tolist()

In [ ]:
nuc_xyz_df = nuc_xyz_df.set_axis(['Col_x', 'Col_y', 'Col_z'], axis=1)
sorted_z = nuc_xyz_df.astype(int).sort_values(by=['Col_z'], ascending=True)
sorted_xz = sorted_z.astype(int).sort_values(by=['Col_x'], ascending=True)
sorted_indices = sorted_xz.astype(int).sort_values(by=['Col_y'], ascending=True).index
xyz_df = xyz_df.reindex(sorted_indices).reset_index(drop=True)

In [ ]:
# ..., and make them into lines
lines = LineMapper(point_column_a='nuc_xyz', point_column_b='soma_xyz')

In [ ]:
# make Neuroglancer link
imgTokyo = ImageLayerConfig(name = 'FANCv4-jp',
                                    source = get_cv_path('Image_Tokyo')['url'])
img = ImageLayerConfig(name = 'FANCv4',
                                    source = get_cv_path('Image')['url'])
seg = SegmentationLayerConfig(name = 'seg_Mar2021_proofreading',
                                    source = get_cv_path('FANC_production_segmentation')['url'])     
nuc_Aug = ImageLayerConfig(name = 'nuc',
                                    source = get_cv_path('nuclei_map_Aug2021')['url'])      
nuc_seg_Aug = ImageLayerConfig(name = 'nuc_seg',
                                    source = get_cv_path('nuclei_seg_Aug2021')['url'])        

ann = AnnotationLayerConfig(name='nuc_soma_Aug2021',
                            mapping_rules=lines,
                            tags=['neuron', 'glia','false_positive','soma_need_check'],
                            active = True)

In [ ]:
view_options = {"layout": "xy"}

memory_options = {"gpuMemoryLimit": 4000000000,
                  "systemMemoryLimit": 8000000000,
                  "concurrentDownloads": 64,
                  "jsonStateServer": "https://global.daf-apis.com/nglstate/api/v1/post"}


sb = StateBuilder(layers=[imgTokyo, seg, img, nuc_Aug, nuc_seg_Aug, ann],
                  resolution=[4.3,4.3,45],
                  view_kws=view_options)

In [ ]:
LINK=[]
k=500
minidfs = [xyz_df.loc[i:i+k-1, :] for i in range(0, len(xyz_df), k)]
for dftmp in minidfs:
    # csb = ChainedStateBuilder([sb, vs])                             
    state = json.loads(sb.render_state(dftmp, return_as='json'))
    state.update(memory_options)
    jsn_id = client.state.upload_state_json(state)
    output = client.state.build_neuroglancer_url(jsn_id, get_cv_path('neuroglancer_base')['url'])
    # output =get_cv_path('neuroglancer_base')['url'] + '?json_url={path}{state_id}'.format(path=get_cv_path('json_server')['url'],state_id=client.state.upload_state_json(state))
    LINK.append(output)

In [ ]:
# save into csv
LINK2 = pd.DataFrame(LINK)
LINK2.to_csv('../Output/links_20210903.csv', index=False, header=False)
# Import this csv file on GoogleSheet, Excel, etc...
# Each link only has 500 lines (nuclei). When you have finished proofreading, create a new json state and save in the 'new link' column
# If you are not sure, just skip it. We can comeback later.

## 2. Extract tags from proofreading results and save those done
After you (roughly) checked each nucleus, save the progress in a csv file. Here, we use function from [this svript](https://github.com/seung-lab/NeuroglancerAnnotationUI/blob/36f03cab5ccff8c52b0faba8beff7ab77398ef48/src/nglui/parser/base.py).

In [ ]:
# run this when you are proofreading for the first time

# df['is_neuron']=""
# df['is_glia']=""
# df['is_false_positive']=""
# df['is_duplicated']=""
# df.to_csv('../Output/proofread_nuc_temp.csv', index=False)

In [ ]:
df_progress = pd.read_csv("../Output/proofread_nuc_temp.csv", header=0)
prfrd1 = pd.read_table("../Output/final_proofreading.tsv", usecols = ['new link'])

In [ ]:
rsplitted = prfrd1['new link'].dropna(how='all').str.rsplit('/', 1)
new_id = list()
for i in rsplitted.index:
    new_id.append(rsplitted.loc[i][1])

print(len(new_id))

In [ ]:
mylist = []

for i in range(len(new_id)):
    state_id = int(new_id[i])
    state = client.state.get_state_json(state_id)

    # extract info from json state
    nuc_tags = parser.tag_dictionary(state,parser.annotation_layers(state)[0])
    anno_lists = parser.line_annotations(state,parser.annotation_layers(state)[0], tags=True)

    temp = pd.DataFrame({'anno_points_A': anno_lists[0],
                     'anno_points_B': anno_lists[1],
                     'anno_tags': anno_lists[2]})

    # convert [] to [0]
    for j in range(len(temp)):
        if (len(temp.iloc[j,2]) == 0) or (len(temp.iloc[j,2]) >= 2): # make dup to 0 for now....
            temp.iloc[j,2] = [0]
            # temp['anno_tags']

    temp['anno_tags'] = np.concatenate(temp['anno_tags']).astype(int)
    temp['anno_tags'] = temp['anno_tags'].replace(nuc_tags)

    mylist.append(temp)


In [ ]:
df_new = pd.concat(mylist).reset_index()

In [ ]:
print('neuron are {}'.format(sum(df_new['anno_tags']=='neuron')))
print('glia are {}'.format(sum(df_new['anno_tags']=='glia')))
print('false_positive are {}'.format(sum(df_new['anno_tags']=='false_positive')))
print('come_back_to_me_later are {}'.format(sum(df_new['anno_tags']==0)))
print('in total {}'.format(len(df_new)))

In [ ]:
df_new.drop("index", axis=1).to_csv('../Output/final_proofread_extracted.csv', index=False) # progress saved

## 3. Make new links for those not proofread yet (and repeat this process)
Some nuclei are not proofread yet, so we made another csv file with links (and repeat this process until there are no nuclei left.)

In [ ]:
df_new2 = df_new[df_new['anno_tags']=='0']
print(len(df_new2))

In [ ]:
xyz_df = df_new2.reindex(columns=['anno_points_A', 'anno_points_B']).reset_index()
nuc_xyz_df = df_new2['anno_points_A'].str.strip('[]').str.split(',',expand=True)
soma_xyz_df = df_new2['anno_points_B'].str.strip('[]').str.split(',',expand=True)
xyz_df['anno_points_A'] = nuc_xyz_df.astype(int).values.tolist()
xyz_df['anno_points_B'] = soma_xyz_df.astype(int).values.tolist()

In [ ]:
lines = LineMapper(point_column_a='anno_points_A', point_column_b='anno_points_B')

In [ ]:
# make Neuroglancer link
imgTokyo = ImageLayerConfig(name = 'FANCv4-jp',
                                    source = get_cv_path('Image_Tokyo')['url'])
img = ImageLayerConfig(name = 'FANCv4',
                                    source = get_cv_path('Image')['url'])
seg = SegmentationLayerConfig(name = 'seg_Mar2021_proofreading',
                                    source = get_cv_path('FANC_production_segmentation')['url'])     
nuc_Aug = ImageLayerConfig(name = 'nuc',
                                    source = get_cv_path('nuclei_map_Aug2021')['url'])      
nuc_seg_Aug = ImageLayerConfig(name = 'nuc_seg',
                                    source = get_cv_path('nuclei_seg_Aug2021')['url'])        

ann = AnnotationLayerConfig(name='nuc_soma_Aug2021',
                            mapping_rules=lines,
                            tags=['neuron', 'glia','false_positive','soma_need_check'],
                            active = True)

In [ ]:
view_options = {"layout": "xy"}

memory_options = {"gpuMemoryLimit": 4000000000,
                  "systemMemoryLimit": 8000000000,
                  "concurrentDownloads": 64,
                  "jsonStateServer": "https://global.daf-apis.com/nglstate/api/v1/post"}


sb = StateBuilder(layers=[imgTokyo, seg, img, nuc_Aug, nuc_seg_Aug, ann],
                  resolution=[4.3,4.3,45],
                  view_kws=view_options)

In [ ]:
LINK=[]
k=500
minidfs = [xyz_df.loc[i:i+k-1, :] for i in range(0, len(xyz_df), k)]
for dftmp in minidfs:
    # csb = ChainedStateBuilder([sb, vs])                             
    state = json.loads(sb.render_state(dftmp, return_as='json'))
    state.update(memory_options)
    jsn_id = client.state.upload_state_json(state)
    output = client.state.build_neuroglancer_url(jsn_id, get_cv_path('neuroglancer_base')['url'])
    LINK.append(output)

In [ ]:
# save into csv
LINK2 = pd.DataFrame(LINK)
LINK2.to_csv('../Output/links_20210928-2.csv', index=False, header=False)

## X. Save final results
When you proofread all the nuclei, run the cells below to save all of your efforts into one single file.

In [ ]:
df_new = pd.read_csv("../Output/final_proofread_extracted.csv", header=0)
df_progress = pd.read_csv("../Output/proofread_nuc_temp.csv", header=0)

In [ ]:
df_new2 = df_new[df_new.anno_tags != '0']

In [ ]:
for i in range(len(df_new2)):
    nuc_loc_temp = df_new2['anno_points_A'].values[i].strip('[]')
    nuc_loc = '(' + nuc_loc_temp + ')'
    nuc_tag = df_new2['anno_tags'].values[i]
    if nuc_loc in df_progress['nuc_xyz'].values:
        idx = df_progress.index[df_progress['nuc_xyz'] == nuc_loc]
        if nuc_tag == 'neuron':
            df_progress.at[idx,'is_neuron'] = 'y'
        if nuc_tag == 'glia':
            df_progress.at[idx,'is_glia'] = 'y'
        if nuc_tag == 'false_positive':
            df_progress.at[idx,'is_false_positive'] = 'y'

In [ ]:
df_come_back_to_me_later = df_progress[~(df_progress.is_neuron=='y') & ~(df_progress.is_glia=='y') & ~(df_progress.is_false_positive=='y')]

In [ ]:
print('neuron are {}'.format(sum(df_progress.is_neuron=='y')))
print('glia are {}'.format(sum(df_progress.is_glia=='y')))
print('false_positive are {}'.format(sum(df_progress.is_false_positive=='y')))
print('come_back_to_me_later are {}'.format(len(df_come_back_to_me_later)))
print('in total {}'.format(len(df_progress)))

In [ ]:
df_progress.to_csv('../Output/proofread_nuc_temp.csv', index=False) # this final output of 1st quality check